In [1]:
import numpy
%run ../../base-notebook.ipynb

2023-07-07 21:42:52 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/contents.txt [746/746] -> "../../ext/contents.txt" [1]
2023-07-07 21:42:52 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_callback.py [8266/8266] -> "../../ext/ml_callback.py" [1]
2023-07-07 21:42:52 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_internal.py [3032/3032] -> "../../ext/ml_internal.py" [1]
2023-07-07 21:42:53 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_io.py [2664/2664] -> "../../ext/ml_io.py" [1]
2023-07-07 21:42:53 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_layer.py [9525/9525] -> "../../ext/ml_layer.py" [1]
2023-07-07 21:42:53 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_plot.py [25391/25391] -> "../../ext/ml_plot.py" [1]
2023-07-07 21:42:53 URL:https://raw.githubusercontent.com/sodeso-nl/ml-utilities/main/ml_util.py [2085/2085] -> "../../ext/ml_util.py" [1]
2023-07-0

In [1]:
extensions_loaded = False
if not extensions_loaded:
    %load_ext autoreload
    %load_ext tensorboard
    extensions_loaded = True

%autoreload 2
%matplotlib inline

In [2]:
import sys

sys.path.append("../../ext")

import ext.ml_callback as mlc
import ext.ml_io as mli
import ext.ml_layer as mll
import ext.ml_plot as mlp
import ext.ml_util as mlu
import ext.ml_data as mld
import ext.ml_view as mlv
import ext.ml_analyze as mla
import ext.ml_shell as mls

import tensorflow as tf
import pandas as pd

import opendatasets as od

import tensorflow_hub as hub

from keras import Model
from keras.optimizers import Adam
from keras.layers import Dense, Input, GlobalAveragePooling1D, Flatten, Activation, TextVectorization, Embedding, LSTM, GRU, Bidirectional, Conv1D, GlobalMaxPooling1D
from keras.losses import binary_crossentropy